In [6]:
import os
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import numpy as np

In [32]:
#make directory to save CSV files
os.makedirs("acs_pums", exist_ok=True)

#ACS 5-Year PUMS data base URL
#only goes back to 2009
dir_url = "https://www2.census.gov/programs-surveys/acs/data/pums/{year}/5-Year/csv_hca.zip"
max_year = 2025
min_year = 2008
#loop through various years to extract CSV files
for year in range(max_year, min_year, -1):
    url = dir_url.format(year=year)
    print(f"Searching for {year} data...")

    #create exception if file is not found
    try:
        response = requests.get(url)
    
        #if response works
        if response.status_code == 200:
            print(f"Downloading CSV for {year}.")
    
            #iterate through directory and grab csv
            with ZipFile(BytesIO(response.content)) as zf:
                for file in zf.namelist():
                    if file.endswith(".csv"):
                        file_name = f"acs_pums_hca_{year}.csv"
                        with open(os.path.join("acs_pums", file_name), "wb") as f_out:
                            f_out.write(zf.read(file))
                        print(f"Saved csv as {file_name}")
        else:
            print(f"No file found for {year}")
    except Exception as e:
        print(f"Failed to download file for {year}")

Searching for 2025 data...
No file found for 2025
Searching for 2024 data...
No file found for 2024
Searching for 2023 data...
Saved csv as acs_pums_hca_2023.csv
Searching for 2022 data...
Saved csv as acs_pums_hca_2022.csv
Searching for 2021 data...
Saved csv as acs_pums_hca_2021.csv
Searching for 2020 data...
Saved csv as acs_pums_hca_2020.csv
Searching for 2019 data...
Saved csv as acs_pums_hca_2019.csv
Searching for 2018 data...
Saved csv as acs_pums_hca_2018.csv
Searching for 2017 data...
Saved csv as acs_pums_hca_2017.csv
Searching for 2016 data...
Saved csv as acs_pums_hca_2016.csv
Searching for 2015 data...
Saved csv as acs_pums_hca_2015.csv
Searching for 2014 data...
Saved csv as acs_pums_hca_2014.csv
Searching for 2013 data...
Saved csv as acs_pums_hca_2013.csv
Searching for 2012 data...
Saved csv as acs_pums_hca_2012.csv
Searching for 2011 data...
Saved csv as acs_pums_hca_2011.csv
Searching for 2010 data...
Saved csv as acs_pums_hca_2010.csv
Searching for 2009 data...
Saved

### The CSV's were quite large and we needed an easier way to work with the data. Here we loop through the CSV's we just saved and copy them to our Postgres server created and deployed on Heroku. Clumn names from the CSV will be used to make a create table statement so an empty table is first created. Then we copy the data into it.

#### APPEND CSV'S TO SINGLE DATAFRAME

Originally wanted this to be the block where we use df.to_sql, but our data is far too large to have it work without failing.

In [39]:
# Folder with ACS CSVs
input_dir = r"C:\Users\nrdee\Documents\Data\Data Science Masters\Data 780\acs_pums"

# Dictionary to collect column names per year
columns_by_year = {}

# Loop through each CSV
for file in os.listdir(input_dir):
    if file.endswith(".csv") and "acs_pums_hca" in file:
        file_path = os.path.join(input_dir, file)
        year = file.split("_")[-1].replace(".csv", "")

        print(f"Reading columns from {file}...")

        # Read just the header row
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line = f.readline().strip()

        # Parse column names
        column_names = header_line.split(',')
        columns_by_year[year] = column_names

# Convert to DataFrame for visualization
col_df = pd.DataFrame.from_dict(columns_by_year, orient='index').sort_index()

# Preview
col_df.head()
col_df.to_csv('columns.csv', index=True)

Reading columns from acs_pums_hca_2009.csv...
Reading columns from acs_pums_hca_2010.csv...
Reading columns from acs_pums_hca_2011.csv...
Reading columns from acs_pums_hca_2012.csv...
Reading columns from acs_pums_hca_2013.csv...
Reading columns from acs_pums_hca_2014.csv...
Reading columns from acs_pums_hca_2015.csv...
Reading columns from acs_pums_hca_2016.csv...
Reading columns from acs_pums_hca_2017.csv...
Reading columns from acs_pums_hca_2018.csv...
Reading columns from acs_pums_hca_2019.csv...
Reading columns from acs_pums_hca_2020.csv...
Reading columns from acs_pums_hca_2021.csv...
Reading columns from acs_pums_hca_2022.csv...
Reading columns from acs_pums_hca_2023.csv...


### Column Reference Table

| Column             | Description                                                                 |
|--------------------|-----------------------------------------------------------------------------|
| ACR                | Lot size (acreage)                                                          |
| ADJHSG             | Housing adjustment factor                                                   |
| ADJINC             | Income adjustment factor                                                    |
| AGS                | Aggregate gross rent as percentage of income                                |
| BDS                | Number of bedrooms                                                          |
| BLD                | Building type (units in structure)                                          |
| DIVISION           | Census division code                                                        |
| FINCP              | Family income                                                              |
| HINCP              | Household income                                                           |
| HUPAC              | Presence and age of children in household                                   |
| NP                 | Number of persons in household                                              |
| NR                 | Number of rooms                                                             |
| NRC                | Number of rooms for computing occupants per room                            |
| PUMA               | Public Use Microdata Area (geographic identifier)                           |
| PUMA_NORMALIZED    | Standardized PUMA column across years                                       |
| REGION             | Census region code                                                          |
| RESMODE            | Residence 1 year ago                                                        |
| RMSP               | Monthly rent                                                               |
| RT                 | Record type (H = housing record)                                            |
| SERIALNO           | Unique housing unit identifier                                              |
| ST                 | State FIPS code                                                             |
| SVAL               | Subsidized housing value                                                    |
| TEN                | Tenure (e.g., owned, rented)                                                |
| TYPE               | Type of unit (e.g., housing unit, group quarters)                           |
| VALP               | Property value                                                              |
| YRBLT              | Year structure was built                                                    |
| acs_year           | Year of the ACS dataset (user-added)                                        |
| acs_file           | Source file name (user-added)                                               |
| BATH               | Number of bathrooms                                                         |
| PUMA00             | PUMA code used in older datasets                                            |
| PUMA10             | PUMA code used in 2010-era datasets                                         |
| PUMA20             | PUMA code used in 2020-era datasets                                         |


In [1]:
#Folder with ACS CSVs
input_dir = r"C:\Users\nrdee\Documents\Data\Data Science Masters\Data 780\acs_pums"

#columns we want to keep
keep_cols = [
'ACR','ADJHSG','ADJINC','AGS','BATH','BDS','BDSP','BLD','DIVISION','FINCP','HINCP','HUPAC','NP','NR','NRC','PUMA',
'PUMA00','PUMA10','PUMA20','PUMA_NORMALIZED','REGION','RESMODE','RMS','RMSP','RT','SERIALNO','ST','STATE','SVAL','TEN','TYPE','TYPEHUGQ',
'VAL','VALP','YBL','YRBLT'
]

#Column normalization mapping
column_standardization = {
    "ST": "STATE",
    "TYPEHUGQ": "TYPE",
    "BDSP": "BDS",
    "RMS": "RMSP",
    "VAL": "VALP",
    "YBL": "YRBLT"
}

#empty list for my dfs
dfs = []

#Loop through each CSV
for file in os.listdir(input_dir):
    if file.endswith(".csv") and "acs_pums_hca" in file:
        file_path = os.path.join(input_dir, file)
        year = file.split("_")[-1].replace(".csv", "")

        #Load CSV
        df = pd.read_csv(file_path)

        #Normalize column names
        df.columns = [column_standardization.get(col, col) for col in df.columns]

        # Normalize PUMA directly
        if "PUMA20" in df:
            df["PUMA_NORMALIZED"] = df["PUMA20"]
        elif "PUMA10" in df:
            df["PUMA_NORMALIZED"] = df["PUMA10"]
        elif "PUMA00" in df:
            df["PUMA_NORMALIZED"] = df["PUMA00"]
        elif "PUMA" in df:
            df["PUMA_NORMALIZED"] = df["PUMA"]
        else: 
            df["PUMA_NORMALIZED"] = pd.NA
            
        #Create columns for additional info on our subsets of data
        df["acs_year"] = int(year)
        df["acs_file"] = file

        #render the df down to only the columns we need
        trimmed_df = df[[col for col in keep_cols + ["acs_year", "acs_file"] if col in df.columns]]

        print(f"Appending {file} to df...")
        #Append df to list
        dfs.append(trimmed_df)

        #Insert into a table (append if exists)
        # df.to_sql("acs_pums", con=engine, if_exists="append", index=False,method='multi',chunksize=10000)

        print(f"Lodaed {file}")

# Combine all into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

Appending acs_pums_hca_2009.csv to df...
Lodaed acs_pums_hca_2009.csv
Appending acs_pums_hca_2010.csv to df...
Lodaed acs_pums_hca_2010.csv
Appending acs_pums_hca_2011.csv to df...
Lodaed acs_pums_hca_2011.csv
Appending acs_pums_hca_2012.csv to df...
Lodaed acs_pums_hca_2012.csv
Appending acs_pums_hca_2013.csv to df...
Lodaed acs_pums_hca_2013.csv
Appending acs_pums_hca_2014.csv to df...
Lodaed acs_pums_hca_2014.csv
Appending acs_pums_hca_2015.csv to df...
Lodaed acs_pums_hca_2015.csv
Appending acs_pums_hca_2016.csv to df...
Lodaed acs_pums_hca_2016.csv
Appending acs_pums_hca_2017.csv to df...
Lodaed acs_pums_hca_2017.csv


C:\Users\nrdee\AppData\Local\Temp\ipykernel_19068\3418735750.py:35: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Appending acs_pums_hca_2018.csv to df...
Lodaed acs_pums_hca_2018.csv


C:\Users\nrdee\AppData\Local\Temp\ipykernel_19068\3418735750.py:35: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Appending acs_pums_hca_2019.csv to df...
Lodaed acs_pums_hca_2019.csv


C:\Users\nrdee\AppData\Local\Temp\ipykernel_19068\3418735750.py:35: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Appending acs_pums_hca_2020.csv to df...
Lodaed acs_pums_hca_2020.csv


C:\Users\nrdee\AppData\Local\Temp\ipykernel_19068\3418735750.py:35: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Appending acs_pums_hca_2021.csv to df...
Lodaed acs_pums_hca_2021.csv
Appending acs_pums_hca_2022.csv to df...
Lodaed acs_pums_hca_2022.csv
Appending acs_pums_hca_2023.csv to df...
Lodaed acs_pums_hca_2023.csv


In [2]:
pd.set_option('display.max_columns', None)

combined_df.head()

,ACR,ADJHSG,ADJINC,AGS,BDS,BLD,DIVISION,FINCP,HINCP,HUPAC,NP,NR,NRC,PUMA,PUMA_NORMALIZED,REGION,RESMODE,RMSP,RT,SERIALNO,STATE,SVAL,TEN,TYPE,VALP,YRBLT,acs_year,acs_file,BATH,PUMA00,PUMA10,PUMA20
0,NaN,1098709,1119794,NaN,2.0,8.0,9,NaN,104100.0,4.0,2,1.0,0.0,6122.0,6122,4,2.0,4.0,H,2005000000005,6,0.0,3.0,1,NaN,5.0,2009,acs_pums_hca_2009.csv,NaN,NaN,NaN,NaN
1,1.0,1098709,1119794,NaN,3.0,2.0,9,74000.0,74000.0,1.0,3,0.0,1.0,1505.0,1505,4,2.0,5.0,H,2005000000015,6,1.0,1.0,1,21.0,7.0,2009,acs_pums_hca_2009.csv,NaN,NaN,NaN,NaN
2,NaN,1098709,1119794,NaN,2.0,9.0,9,NaN,150000.0,4.0,2,1.0,0.0,8101.0,8101,4,1.0,3.0,H,2005000000033,6,0.0,1.0,1,21.0,4.0,2009,acs_pums_hca_2009.csv,NaN,NaN,NaN,NaN
3,1.0,1098709,1119794,NaN,3.0,2.0,9,46800.0,46800.0,2.0,3,0.0,1.0,8005.0,8005,4,1.0,5.0,H,2005000000034,6,0.0,3.0,1,NaN,3.0,2009,acs_pums_hca_2009.csv,NaN,NaN,NaN,NaN
4,1.0,1098709,1119794,NaN,2.0,3.0,9,NaN,64000.0,4.0,1,0.0,0.0,2405.0,2405,4,1.0,5.0,H,2005000000044,6,1.0,1.0,1,21.0,5.0,2009,acs_pums_hca_2009.csv,NaN,NaN,NaN,NaN


In [3]:
combined_df.dtypes

ACR                float64
ADJHSG               int64
ADJINC               int64
AGS                float64
BDS                float64
BLD                float64
DIVISION             int64
FINCP              float64
HINCP              float64
HUPAC              float64
NP                   int64
NR                 float64
NRC                float64
PUMA               float64
PUMA_NORMALIZED      int64
REGION               int64
RESMODE            float64
RMSP               float64
RT                  object
SERIALNO            object
STATE                int64
SVAL               float64
TEN                float64
TYPE                 int64
VALP               float64
YRBLT              float64
acs_year             int64
acs_file            object
BATH               float64
PUMA00             float64
PUMA10             float64
PUMA20             float64
dtype: object

#### Created table in Postgres before sending my data

In [4]:
#Auto-generate CREATE TABLE statement from my columns
def generate_create_table_sql(df, table_name):
    type_map = {
        'int64': 'INT',
        'Int64': 'INT',
        'float64': 'FLOAT',
        'object': 'TEXT'
    }

    columns = []
    for col, dtype in df.dtypes.items():

        clean_col = col.strip().replace(" ", "_").replace("-", "_").lower()
        
        #Ensure serial num is text
        if clean_col == 'serialno':
            sql_type = 'TEXT'
        else:
            sql_type = type_map.get(str(dtype), 'TEXT')

        columns.append(f'"{clean_col}" {sql_type}') 
    
    column_defs = ",\n    ".join(columns)
    return f'CREATE TABLE IF NOT EXISTS {table_name} (\n    {column_defs}\n);'

#Generate and print the SQL
table_sql = generate_create_table_sql(combined_df, "acs_pums")
print(table_sql)

CREATE TABLE IF NOT EXISTS acs_pums (
    "acr" FLOAT,
    "adjhsg" INT,
    "adjinc" INT,
    "ags" FLOAT,
    "bds" FLOAT,
    "bld" FLOAT,
    "division" INT,
    "fincp" FLOAT,
    "hincp" FLOAT,
    "hupac" FLOAT,
    "np" INT,
    "nr" FLOAT,
    "nrc" FLOAT,
    "puma" FLOAT,
    "puma_normalized" INT,
    "region" INT,
    "resmode" FLOAT,
    "rmsp" FLOAT,
    "rt" TEXT,
    "serialno" TEXT,
    "state" INT,
    "sval" FLOAT,
    "ten" FLOAT,
    "type" INT,
    "valp" FLOAT,
    "yrblt" FLOAT,
    "acs_year" INT,
    "acs_file" TEXT,
    "bath" FLOAT,
    "puma00" FLOAT,
    "puma10" FLOAT,
    "puma20" FLOAT
);


#### Iterate through data in chunks. Unable to load data all at once.

In [7]:
#Path for our temp CSV that will be copied into our db
temp_path = "temp_acs_pums.csv"
combined_df.to_csv(temp_path, index=False, header=True)

#set our chunk size
chunk_size = 500000

#Split and load each chunk
for i, chunk in enumerate(np.array_split(combined_df, len(combined_df) // chunk_size + 1)):
    if i = 12 then:
        temp_chunk_path = f"temp_chunk_{i}.csv"
        chunk.to_csv(temp_chunk_path, index=False)
    
        try:
            #New connection per chunk
            conn = psycopg2.connect(
                dbname="d9f89h4ju1lleh",
                user="ufnbfacj9c7u80",
                password="pa129f8c5adad53ef2c90db10cce0c899f8c7bdad022cca4e85a8729b19aad68d",
                host="ceq2kf3e33g245.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com",
                port="5432"
            )
            
            cur = conn.cursor()
    
            with open(temp_chunk_path, "r", encoding="utf-8") as f:
                cur.copy_expert("COPY acs_pums FROM STDIN WITH CSV HEADER", f)
    
            conn.commit()
            cur.close()
            conn.close()
            print(f"Inserted chunk {i}")
        except Exception as e:
            print(f"Chunk {i} failed:", e)
    else:
        print(f"SKIPPED {i}"

    os.remove(temp_chunk_path)

C:\Users\nrdee\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Inserted chunk 0
Inserted chunk 1
Inserted chunk 2
Inserted chunk 3
Inserted chunk 4
Inserted chunk 5
Inserted chunk 6
Inserted chunk 7
Inserted chunk 8
Inserted chunk 9
Inserted chunk 10
Inserted chunk 11
Chunk 12 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
invalid socket

Inserted chunk 13
Inserted chunk 14
Inserted chunk 15
Inserted chunk 16
Inserted chunk 17
Inserted chunk 18
Inserted chunk 19
Inserted chunk 20
Inserted chunk 21
Inserted chunk 22
